In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-10"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
735,2024-07-10,Venezuela Superliga,20:00,Llaneros,Toros de Aragua,1.33,3.12,162.5,1.85,1.85,0.0,0.00,0.00,SbEKbKUj,0.751880,0.320513,0.540541,0.540541,0.072393,200.980,50.630268,0.251917,1.2,1.643168,1.369306,244.94,2.724,0.720472,0.264490,-17.0,355.382,143.812457,0.404670,0.0,0.000000,NaN,358.11,4.636,1.973482,0.425686,-80.0,74,69,3.31,5.19,190.256,373.038,0.568863,0.000000,NaN,0.70,0.140,2.357143,0.000000,0.0,0.000000,-5.00,-1.000,-2.120000,0.000000,0.0,0.000000
736,2024-07-10,Venezuela Superliga,20:00,Marinos,Piratas de La Guaira,1.36,2.97,150.5,1.81,1.86,0.0,0.00,0.00,r7d7MNMc,0.735294,0.336700,0.552486,0.537634,0.071994,99.654,12.998915,0.130440,0.6,1.341641,2.236068,88.20,1.362,0.151723,0.111398,-24.0,158.828,51.777367,0.325996,1.8,1.643168,0.912871,158.41,2.038,0.778184,0.381837,16.0,70,73,1.26,2.17,96.692,190.328,0.525839,0.019267,NaN,-2.51,-0.502,-0.717131,0.758957,0.6,-0.158957,-2.94,-0.588,-3.350340,0.000000,0.0,0.000000
737,2024-07-10,Vietnã Vba,09:30,Nhatrang Dolphins,Can Tho Catfish,1.39,2.84,163.5,1.85,1.85,0.0,0.00,0.00,dMWhDBE6,0.719424,0.352113,0.540541,0.540541,0.071537,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,85,0,1.55,0.00,0.000,0.000,0.484778,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
738,2024-07-10,Eua Clássico Da Califórnia,20:00,Los Angeles Lakers,Miami Heat,2.68,1.46,175.5,1.88,1.90,0.0,0.00,0.00,OfLyxuYO,0.373134,0.684932,0.531915,0.526316,0.058066,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.416749,0.007483,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
739,2024-07-10,Eua Clássico Da Califórnia,23:00,Golden State Warriors,Sacramento Kings,1.20,4.43,183.5,1.80,1.94,0.0,0.00,0.00,2LjndrJt,0.833333,0.225734,0.555556,0.515464,0.059067,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.811352,0.052938,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
740,2024-07-10,Porto Rico Bsn,21:00,Leones De Ponce,Indios de Mayaguez,1.62,2.20,177.5,1.86,1.84,0.0,0.00,0.00,QyqCwtLG,0.617284,0.454545,0.537634,0.543478,0.071829,192.642,45.948358,0.238517,2.4,1.341641,0.559017,197.80,2.074,0.612642,0.295391,29.0,184.380,35.878899,0.194592,0.6,1.341641,2.236068,195.94,2.052,0.106630,0.051964,-16.0,92,101,2.15,1.94,164.964,185.640,0.214724,0.007644,NaN,2.29,0.458,1.353712,0.554663,0.7,0.145337,-5.00,-1.000,-1.200000,0.441228,0.0,-0.441228
741,2024-07-10,Europa Eurobasket Sub,12:00,Hungria U20 F,Polónia U20 F,1.40,2.80,132.5,1.86,1.84,0.0,0.00,0.00,Mm8E2bBC,0.714286,0.357143,0.537634,0.543478,0.071429,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,86,62,1.74,10.00,0.000,0.000,0.471405,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
742,2024-07-10,Eua Liga De Verão Da Nba,22:00,Utah J

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
